In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import json
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple
from enum import Enum

print("Advanced Multi-Agent System Initialized")

Advanced Multi-Agent System Initialized


In [15]:
# ============ ENUMS ============
class Priority(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class Sentiment(Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

# ============ MEMORY SYSTEM ============
@dataclass
class Memory:
    messages: List[Dict] = field(default_factory=list)
    customer_profile: Dict = field(default_factory=dict)
    escalation_history: List[Dict] = field(default_factory=list)
    max_history: int = 50
    
    def add(self, role: str, content: str, metadata: Optional[Dict] = None):
        entry = {
            "role": role,
            "content": content,
            "time": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        self.messages.append(entry)
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
    
    def get_context(self, last_n: int = 5) -> str:
        context = ""
        for m in self.messages[-last_n:]:
            context += f"{m['role']}: {m['content']}\n"
        return context
    
    def update_profile(self, key: str, value):
        self.customer_profile[key] = value
    
    def record_escalation(self, reason: str, agent: str):
        self.escalation_history.append({
            "time": datetime.now().isoformat(),
            "reason": reason,
            "agent": agent
        })

# ============ INTENT CLASSIFICATION AGENT ============
class IntentAgent:
    def __init__(self):
        self.intent_patterns = {
            "refund": ["refund", "money back", "return money", "get my money"],
            "cancellation": ["cancel", "stop subscription", "end service", "discontinue"],
            "billing": ["invoice", "charge", "bill", "payment", "wrong amount", "overcharged"],
            "technical_support": ["not working", "broken", "error", "bug", "crash", "issue"],
            "account_access": ["can't login", "forgot password", "locked out", "reset"],
            "product_inquiry": ["how does", "what is", "explain", "feature"],
            "complaint": ["disappointed", "terrible", "worst", "never again"],
            "upgrade": ["upgrade", "premium", "pro version", "better plan"],
            "shipping": ["delivery", "tracking", "shipment", "hasn't arrived"],
            "general_help": ["help", "question", "need assistance"]
        }
    
    def classify(self, message: str) -> Tuple[str, Priority]:
        text = message.lower()
        matched_intents = []
        
        for intent, keywords in self.intent_patterns.items():
            if any(kw in text for kw in keywords):
                matched_intents.append(intent)
        
        if not matched_intents:
            return "general", Priority.LOW
        
        primary_intent = matched_intents[0]
        
        # Determine priority
        if primary_intent in ["refund", "complaint", "account_access"]:
            priority = Priority.HIGH
        elif primary_intent in ["billing", "cancellation", "technical_support"]:
            priority = Priority.MEDIUM
        else:
            priority = Priority.LOW
        
        return primary_intent, priority

# ============ SENTIMENT ANALYSIS AGENT ============
class SentimentAgent:
    def __init__(self):
        self.angry_keywords = ["angry", "furious", "outraged", "terrible", "worst", "pathetic", "ridiculous"]
        self.frustrated_keywords = ["frustrated", "annoyed", "disappointed", "upset", "still waiting"]
        self.positive_keywords = ["thank", "great", "appreciate", "perfect", "excellent"]
    
    def analyze(self, message: str) -> Sentiment:
        text = message.lower()
        
        # Check for caps lock (indicates strong emotion)
        caps_ratio = sum(1 for c in message if c.isupper()) / max(len(message), 1)
        
        if any(kw in text for kw in self.angry_keywords) or caps_ratio > 0.5:
            return Sentiment.ANGRY
        elif any(kw in text for kw in self.frustrated_keywords):
            return Sentiment.FRUSTRATED
        elif any(kw in text for kw in self.positive_keywords):
            return Sentiment.SATISFIED
        
        return Sentiment.NEUTRAL

# ============ ESCALATION AGENT ============
class EscalationAgent:
    def __init__(self):
        self.escalation_triggers = {
            "repeated_contact": 3,
            "angry_sentiment_threshold": 2,
            "high_value_customer": True
        }
    
    def should_escalate(self, memory: Memory, sentiment: Sentiment, 
                       priority: Priority, customer_value: str) -> Tuple[bool, str]:
        reasons = []
        
        # Check if customer is angry
        if sentiment == Sentiment.ANGRY:
            reasons.append("Customer shows signs of extreme dissatisfaction")
        
        # Check if critical priority
        if priority == Priority.CRITICAL or priority == Priority.HIGH:
            angry_count = sum(1 for m in memory.messages[-5:] 
                            if m.get('metadata', {}).get('sentiment') in ['angry', 'frustrated'])
            if angry_count >= 2:
                reasons.append("Multiple frustrated interactions detected")
        
        # Check customer value
        if customer_value == "premium" and priority != Priority.LOW:
            reasons.append("Premium customer requires priority handling")
        
        # Check escalation history
        if len(memory.escalation_history) > 0:
            last_escalation = memory.escalation_history[-1]
            reasons.append("Previous escalation on record")
        
        should_escalate = len(reasons) > 0
        escalation_reason = "; ".join(reasons) if reasons else "No escalation needed"
        
        return should_escalate, escalation_reason

# ============ REPLY GENERATION AGENT ============
class ReplyAgent:
    def __init__(self):
        self.templates = {
            "refund": {
                Sentiment.ANGRY: "I completely understand your frustration, and I sincerely apologize. I'm prioritizing your refund request right now. Please provide your order ID, and I'll process this immediately.",
                Sentiment.FRUSTRATED: "I'm sorry to hear about this issue. I'll help you get your refund processed quickly. Could you please share your order ID?",
                Sentiment.NEUTRAL: "I can help you with a refund. Please provide your order ID so I can look into this for you."
            },
            "cancellation": {
                Sentiment.ANGRY: "I understand you want to cancel, and I'll make this as smooth as possible. Before I proceed, is there anything specific that led to this decision? I'd like to help if I can.",
                Sentiment.FRUSTRATED: "I can assist with canceling your subscription. May I ask what prompted this decision? Your feedback is valuable to us.",
                Sentiment.NEUTRAL: "I can help you cancel your subscription. Please provide your registered email address."
            },
            "billing": {
                Sentiment.ANGRY: "I sincerely apologize for any billing errors. This is unacceptable, and I'm going to review your account immediately. Please share your invoice number.",
                Sentiment.FRUSTRATED: "I'm sorry about the billing issue. Let me investigate this right away. Could you provide your invoice number?",
                Sentiment.NEUTRAL: "I'll help you resolve this billing matter. Please share your invoice number for verification."
            },
            "technical_support": {
                Sentiment.ANGRY: "I'm very sorry you're experiencing technical difficulties. This must be incredibly frustrating. Let me get our technical team on this immediately. What specific issue are you encountering?",
                Sentiment.FRUSTRATED: "I apologize for the technical trouble. Let me help you resolve this. Can you describe what's happening?",
                Sentiment.NEUTRAL: "I'm here to help with your technical issue. Could you describe the problem you're experiencing?"
            },
            "account_access": {
                Sentiment.ANGRY: "I understand how frustrating being locked out of your account is. I'm going to help you regain access right now. What's your registered email address?",
                Sentiment.NEUTRAL: "I can help you regain access to your account. Please provide your registered email address for verification."
            },
            "complaint": {
                Sentiment.ANGRY: "I'm truly sorry to hear about your experience. Your feedback is extremely important to us. I want to make this right. Can you tell me more about what happened?",
                Sentiment.FRUSTRATED: "I apologize that we didn't meet your expectations. I'd like to understand what went wrong so I can help. Could you share more details?"
            },
            "upgrade": {
                Sentiment.NEUTRAL: "Great! I'd be happy to help you upgrade. Let me walk you through our premium options and find the best fit for your needs."
            },
            "shipping": {
                Sentiment.FRUSTRATED: "I apologize for the delay. Let me track your order right away. Please provide your order number.",
                Sentiment.NEUTRAL: "I'll help you track your shipment. Could you provide your order number?"
            }
        }
    
    def create_reply(self, intent: str, sentiment: Sentiment, 
                    escalated: bool, customer_value: str) -> str:
        # Handle escalation
        if escalated:
            return "I'm connecting you with a senior specialist who can provide immediate assistance. They'll be with you shortly and will have full context of your situation."
        
        # Get template based on intent and sentiment
        intent_templates = self.templates.get(intent, {})
        
        # Try to get sentiment-specific template
        reply = intent_templates.get(sentiment)
        
        # Fallback to neutral sentiment
        if not reply:
            reply = intent_templates.get(Sentiment.NEUTRAL)
        
        # Final fallback
        if not reply:
            if sentiment == Sentiment.ANGRY:
                reply = "I sincerely apologize for your experience. I want to help resolve this immediately. Could you provide more details?"
            else:
                reply = "Thank you for reaching out. I'm here to help. Could you please provide more details about your request?"
        
        # Add premium customer acknowledgment
        if customer_value == "premium" and not escalated:
            reply = f"As one of our valued premium members, {reply}"
        
        return reply

# ============ CONTEXT ENRICHMENT AGENT ============
class ContextAgent:
    def __init__(self):
        self.customer_database = {
            "user123": {"name": "John Doe", "tier": "premium", "orders": 15},
            "user456": {"name": "Jane Smith", "tier": "basic", "orders": 2},
            "user789": {"name": "Bob Johnson", "tier": "premium", "orders": 47}
        }
    
    def enrich_context(self, memory: Memory, message: str) -> Dict:
        # Extract potential user identifiers from message
        customer_id = self._extract_customer_id(message)
        
        enrichment = {
            "customer_tier": "basic",
            "interaction_count": len(memory.messages),
            "previous_issues": [],
            "customer_lifetime_value": "standard"
        }
        
        if customer_id and customer_id in self.customer_database:
            customer_data = self.customer_database[customer_id]
            enrichment["customer_tier"] = customer_data["tier"]
            enrichment["customer_name"] = customer_data["name"]
            enrichment["order_count"] = customer_data["orders"]
            enrichment["customer_lifetime_value"] = "high" if customer_data["orders"] > 10 else "standard"
        
        return enrichment
    
    def _extract_customer_id(self, message: str) -> Optional[str]:
        # Simple extraction - in reality, this would be more sophisticated
        for user_id in ["user123", "user456", "user789"]:
            if user_id in message.lower():
                return user_id
        return None

# ============ ANALYTICS AGENT ============
class AnalyticsAgent:
    def __init__(self):
        self.metrics = {
            "total_interactions": 0,
            "intent_distribution": {},
            "sentiment_distribution": {},
            "escalation_rate": 0,
            "escalations": 0
        }
    
    def record_interaction(self, intent: str, sentiment: Sentiment, escalated: bool):
        self.metrics["total_interactions"] += 1
        
        # Track intent distribution
        self.metrics["intent_distribution"][intent] = \
            self.metrics["intent_distribution"].get(intent, 0) + 1
        
        # Track sentiment distribution
        sentiment_key = sentiment.value
        self.metrics["sentiment_distribution"][sentiment_key] = \
            self.metrics["sentiment_distribution"].get(sentiment_key, 0) + 1
        
        # Track escalations
        if escalated:
            self.metrics["escalations"] += 1
        
        self.metrics["escalation_rate"] = \
            (self.metrics["escalations"] / self.metrics["total_interactions"]) * 100
    
    def get_report(self) -> Dict:
        return {
            "summary": self.metrics,
            "generated_at": datetime.now().isoformat()
        }

# ============ MASTER COORDINATOR ============
class MasterCoordinator:
    def __init__(self):
        self.intent_agent = IntentAgent()
        self.sentiment_agent = SentimentAgent()
        self.escalation_agent = EscalationAgent()
        self.reply_agent = ReplyAgent()
        self.context_agent = ContextAgent()
        self.analytics_agent = AnalyticsAgent()
        self.memory = Memory()
    
    def process_message(self, message: str) -> Dict:
        # Step 1: Enrich context
        context = self.context_agent.enrich_context(self.memory, message)
        
        # Step 2: Classify intent
        intent, priority = self.intent_agent.classify(message)
        
        # Step 3: Analyze sentiment
        sentiment = self.sentiment_agent.analyze(message)
        
        # Step 4: Check for escalation
        customer_tier = context.get("customer_tier", "basic")
        should_escalate, escalation_reason = self.escalation_agent.should_escalate(
            self.memory, sentiment, priority, customer_tier
        )
        
        # Step 5: Generate reply
        reply = self.reply_agent.create_reply(intent, sentiment, should_escalate, customer_tier)
        
        # Step 6: Store in memory
        metadata = {
            "intent": intent,
            "priority": priority.value,
            "sentiment": sentiment.value,
            "escalated": should_escalate
        }
        self.memory.add("user", message, metadata)
        self.memory.add("agent", reply, metadata)
        
        # Step 7: Record escalation if needed
        if should_escalate:
            self.memory.record_escalation(escalation_reason, "MasterCoordinator")
        
        # Step 8: Record analytics
        self.analytics_agent.record_interaction(intent, sentiment, should_escalate)
        
        # Build response
        return {
            "intent": intent,
            "priority": priority.value,
            "sentiment": sentiment.value,
            "customer_tier": customer_tier,
            "escalated": should_escalate,
            "escalation_reason": escalation_reason if should_escalate else None,
            "reply": reply,
            "context": context
        }
    
    def get_analytics(self) -> Dict:
        return self.analytics_agent.get_report()


# ============ DEMONSTRATION ============
def run_demo():
    agent = MasterCoordinator()
    
    test_scenarios = [
        "I want to cancel my subscription. I'm user123.",
        "This is RIDICULOUS! I've been overcharged AGAIN!",
        "My account is locked and I can't login. Please help.",
        "The app keeps crashing whenever I try to upload a file.",
        "I'd like to upgrade to the premium plan. user789 here.",
        "I'm extremely disappointed with your service. This is the worst experience ever!",
        "My package hasn't arrived yet. Order #12345.",
        "I need a refund immediately. This product doesn't work as advertised!",
        "Can you explain how the advanced analytics feature works?",
        "Thank you for your help yesterday. Everything is working great now!",
        "I'm user456 and I still haven't received my refund from last week. This is getting frustrating.",
    ]
    
    print("\n"  "="*80)
    print("MULTI-AGENT CUSTOMER SERVICE SYSTEM - LIVE DEMO")
    print("="*80 + "\n")
    
    for i, msg in enumerate(test_scenarios, 1):
        print(f"\n{'─'*80}")
        print(f"INTERACTION #{i}")
        print(f"{'─'*80}")
        print(f"🗣️  USER: {msg}\n")
        
        response = agent.process_message(msg)
        
        print(f"📊 ANALYSIS:")
        print(f"   Intent: {response['intent']}")
        print(f"   Priority: {response['priority']}")
        print(f"   Sentiment: {response['sentiment']}")
        print(f"   Customer Tier: {response['customer_tier']}")
        print(f"   Escalated: {'⚠️  YES' if response['escalated'] else '✓ NO'}")
        if response['escalated']:
            print(f"   Escalation Reason: {response['escalation_reason']}")
        
        print(f"\n🤖 AGENT REPLY:\n   {response['reply']}\n")
    
    # Print analytics report
    print("\n" + "="*80)
    print("ANALYTICS REPORT")
    print("="*80)
    analytics = agent.get_analytics()
    print(json.dumps(analytics, indent=2))
    print("\n")

if __name__ == "__main__":
    run_demo()


=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
MULTI-AGENT CUSTOMER SERVICE SYSTEM - LIVE DEMO


────────────────────────────────────────────────────────────────────────────────
INTERACTION #1
────────────────────────────────────────────────────────────────────────────────
🗣️  USER: I want to cancel my subscription. I'm user123.

📊 ANALYSIS:
   Intent: cancellation
   Priority: medium
   Sentiment: neutral
   Customer Tier: premium
   Escalated: ⚠️  YES
   Escalation Reason: Premium customer requires priority handling

🤖 AGENT REPLY:
   I'm connecting you with a senior specialist who can provide immediate assistance. They'll be with you shortly and will have full context of your situation.


────────────────────────────────────────────────────────────────────────────────
INTERACTION #2
────────────────────────────────────────────────────────────────────────────────
🗣️  USER